In [104]:
import pandas as pd
from pandas import json_normalize
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
from tqdm import tqdm
import time 

In [105]:
# loading cluster dataset

In [106]:
def connect_spotipy():
    with open("secrets.txt","r") as f:
        string = f.read()
    
    secrets_dict={}
    
    for line in string.split('\n'):
        if len(line) > 0:
            #print(line.split(':'))
            secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
            

    #Initialize SpotiPy with user credentials
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                            client_secret=secrets_dict['clientsecret']))
    return sp

In [107]:
with open("temp_data/top100.csv", 'rb') as f:
    df1 = pd.read_csv(f)

df1.head(2)

,Unnamed: 0,Song,Artist
0,0,Rich Men North of Richmond,Oliver Anthony Music
1,1,Paint The Town Red,Doja Cat


In [108]:
with open("temp_data/clustered_songs.csv", 'rb') as f:
    df2 = pd.read_csv(f)

df2.head(2)

,Unnamed: 0,track.id,track.name,artist_name,track.external_urls.spotify,artist_dict,track.album.id,track.album.name,track.album.images,track.album.release_date,cluster
0,0,31nfdEooLEq7dn3UMcIeB5,Vois sur ton chemin - Techno Mix,BENNETT,https://open.spotify.com/track/31nfdEooLEq7dn3...,{0: {'external_urls': {'spotify': 'https://ope...,79Cyc8GRWnLyjdJSMyJ0dB,Vois sur ton chemin (Techno Mix),"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2023-08-04,1
1,1,0gMTEHzNIyvxikxyUFFJxO,Summertime,Kontra K,https://open.spotify.com/track/0gMTEHzNIyvxikx...,{0: {'external_urls': {'spotify': 'https://ope...,3d5e4tp3t5zTGu44NM88LQ,Summertime,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2023-08-10,1


In [109]:
# def return_top_list(tracks, song):
#     print(f"There are many songs with the name {song}. Which one do you want?")
    
#     for i in range(len(tracks)):
#         song_url = tracks.iloc[i]['external_urls.spotify']  
#         song_artist = tracks.iloc[i]['artists'][0]['name']
#         track_name = tracks.iloc[0]['name']

#         print(f""" 
#         Song {i+1}: {track_name} from {song_artist} -> {song_url}                 
#         """)
          
#     try:
#         selection = int(input('Which one do you want?')) - 1
#         return tracks.iloc[selection]['id']
#     except (ValueError, IndexError):
#         print("Invalid selection, please try again.")
#         return None

In [115]:
def song_recommender(df1, df2):
    sp = connect_spotipy()
    song_id = None
    
    while song_id is None:
        song = input("Please enter the name of a song: ")
        sp = connect_spotipy()
        request = sp.search(q='track:' + song, type='track')
        tracks = pd.json_normalize(request["tracks"]['items'])
        if not tracks.empty:
            song_id = tracks.iloc[0]['id']
            song_url = tracks.iloc[0]['external_urls.spotify']
            song_artist = tracks.iloc[0]['artists'][0]['name']
            track_name = tracks.iloc[0]['name']
            print(f"""end=
Searching for song thats matches too {track_name} by {song_artist}:                  
                  """)
        else:
            print("Song not found in Spotify.")
            return
        
    audio_features_list = sp.audio_features(song_id)
    audio_features_df = pd.json_normalize(audio_features_list)
    audio_features_df = audio_features_df[['duration_ms', 'danceability','energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness','instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]    
    
    with open("model_data/standardScaler.p", 'rb') as f:
        loaded_scaler = pickle.load(f)
        X = loaded_scaler.transform(audio_features_df)
    with open("model_data/kmean_model.p", 'rb') as f:
        loaded_model = pickle.load(f)
        clusters = loaded_model.predict(X)
    
    matching_song = df1[(df1['Song'].str.lower() == song.lower())]
    if not matching_song.empty:
        print(f"The song '{matching_song['Song'].values[0]}' by {matching_song['artist'].values[0]} matches your input.")
        recommended_song = df1[df1['song'].str.lower() != song].sample(1)
        print(f"Here's a recommended song: '{recommended_song['song'].values[0]}' by {recommended_song['artist'].values[0]}")
        results2 = sp.search(q='track:' + recommended_song['song'].values[0], type='track')
        track2 = pd.json_normalize(results2["tracks"]["items"])
        song_url = track2.iloc[0]['external_urls.spotify']
        print(f"Here's a url of the song: {song_url}")
    else:
        cluster_df = df2[df2['cluster'] == clusters[0]]
        recommended_song2 = cluster_df[cluster_df['track.name'].str.lower() != song].sample(1)
        print(f"Song Recommendation: '{recommended_song2['track.name'].values[0]}' by {recommended_song2['artist_name'].values[0]}")
        results3 = sp.search(q='track:' + recommended_song2['track.name'].values[0], type='track')
        track3 = pd.json_normalize(results3["tracks"]["items"])
        song_url = track3.iloc[0]['external_urls.spotify']
        print(f"Song URL: {song_url}")

In [116]:

song_recommender(df1, df2)

end=
Searching for song thats matches too Somebody to Love by Jefferson Airplane:                  
                  
Song Recommendation: 'Emergency' by Sofi Tukker


C:\Users\Domen\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


Song URL: https://open.spotify.com/track/4amltxLIfFmtYEvZbdgDqO


currently really bad suggestions and the code breaks easily. 